# Debug SqueezeNet v1.3 (Simple Task) OpenCL implement with PyOpenCL and PyTorch
Partial code are copied heavily from https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py  
SqueezeNet Paper:https://arxiv.org/abs/1602.07360  
SqueezeNet 1.1 model from https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1   
SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters than SqueezeNet 1.0, without sacrificing accuracy.

TEST DE IMPLEMENTACIÓN CONV3x3

In [1]:
#some set up
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
import math
import time
from time import sleep, perf_counter as pc
from matplotlib.pyplot import imshow
%matplotlib inline

## Aquí tenemos el medidor de pytorch

- En esta aproximacion tengo en cuenta el tiempo de definición de capas y la carga de pesos porque se supone que entre fire y fire estas operaciones tiene que hacerse


In [2]:
canales_iniciales=4 #input_channels
canales_contraidos=2 #filter_size
canales_finales= canales_iniciales
acumulado_pytorch=0
idea=True
count=100
tamanyo=5 #input_size
squeeze_activation = nn.ReLU(inplace=True)

for i in range(count):

    imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))
    weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
      
    tic=pc()
    squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
    squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
    squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))    
    imagen1  = torch.from_numpy(imagen).float()
    
    salida1=squeeze1(imagen1)
    salida1_activation=squeeze_activation(salida1)

    salida1_a_numpy=salida1_activation.detach().numpy()
    
    toc=pc()
    
    acumulado_pytorch=toc-tic+acumulado_pytorch
    
print(salida1_a_numpy)
print ("tiempo en segundos con pytorch= ", acumulado_pytorch/count)

[[[[ 50.  70.  88.  87. 124.]
   [ 94. 140.  57. 138. 102.]
   [ 78.  84.  96.  91. 119.]
   [ 41. 108.  79.  77.  81.]
   [ 66.  79. 121.  70.  62.]]

  [[ 29. 101.  55.  86.  71.]
   [ 86. 116.  37.  86.  75.]
   [ 67.  95. 138.  28.  64.]
   [ 30.  53.  99.  92.  83.]
   [ 90.  71. 161.  58.  88.]]]]
tiempo en segundos con pytorch=  0.00011299011999881259


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


## Veamos ahora solo conv3x3 con opencl

In [3]:
# OpenCL setup
import pyopencl as cl
import sys
sys.path.append('../common')
import deviceinfo
from time import time

#wksp = '../device/v1.3/conv3x3'


#### Step0-A: compilation for emulation

In [40]:
%%bash
aoc -march=emulator ../device/v1.3/conv3x3/conv3x3_NDRange.cl -o ../device/v1.3/conv3x3/bin_em/conv3x3_NDRange.aocx
aoc -march=emulator ../device/v1.3/conv3x3/conv3x3_ST.cl -o ../device/v1.3/conv3x3/bin_em/conv3x3_ST.aocx

aoc: OpenCL kernel compilation completed successfully.
aoc: Linking Object files....
aoc: Compiling for Emulation ....
aoc: OpenCL kernel compilation completed successfully.
aoc: Linking Object files....
aoc: Compiling for Emulation ....


#### Step0-B: compilation for simulation

In [41]:
%%bash
aoc -march=simulator -v -ghdl ../device/v1.3/conv3x3/conv3x3_NDRange.cl -o ../device/v1.3/conv3x3/bin_sim/conv3x3_NDRange.aocx -board=a10gx
aoc -march=simulator -v -ghdl ../device/v1.3/conv3x3/conv3x3_ST.cl -o ../device/v1.3/conv3x3/bin_sim/conv3x3_ST.aocx -board=a10gx


aoc: Environment checks completed successfully.
Quartus location: /home/joerock/intelFPGA_pro/21.1/quartus/bin/quartus_sh
aoc: Cached files in /var/tmp/aocl/joerock may be used to reduce compilation time
aoc: Selected target board package /home/joerock/intelFPGA_pro/21.1/hld/board/a10_ref
aoc: Selected target board a10gx
aoc: Running OpenCL parser....
aoc: OpenCL parser completed 
aoc: Linking Object files....
aoc: Optimizing and doing static analysis of code...
aoc: Linking with IP library ...
aoc: Checking if memory usage is larger than 100%...
aoc: Memory usage is not above 100.
aoc: First stage compilation completed successfully.
aoc: Compiling for Simulator.
Quartus location: /home/joerock/intelFPGA_pro/21.1/quartus/bin/quartus_sh
Creating simulation system...
Generating simulation system...
Compiling simulation...
aoc: Simulation generation done!
Simulator flow is successful.
To execute simulator, invoke host with 
	env CL_CONTEXT_MPSIM_DEVICE_INTELFPGA=1 <host_program>
aoc: Envi

#### Step1: OpenCL preparation

In [42]:
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[0])])
queue = cl.CommandQueue(context)

context

<pyopencl.Context at 0x687a4a8 on <pyopencl.Device '12th Gen Intel(R) Core(TM) i7-12650H' on 'Intel(R) OpenCL' at 0x4f7fab8>>

#### Step 2: creat kernels
Creat & build program

In [43]:
wksp = '../device/v1.3/conv3x3/'

file_dir = wksp + 'conv3x3_NDRange.cl'

kernelSource = open(file_dir).read()
program_NDR = cl.Program(context, kernelSource).build()

file_dir = wksp + 'conv3x3_ST.cl'

kernelSource = open(file_dir).read()
program_ST = cl.Program(context, kernelSource).build()

/home/joerock/anaconda3/envs/TFM/lib/python3.8/site-packages/pyopencl/__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Creat kernels

In [44]:
conv3x3_NDR = program_NDR.conv2d3x3
conv3x3_NDR.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

conv3x3_ST = program_ST.conv2d3x3
conv3x3_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])


#### OpenCL kernel: conv3x3_NDRange.cl

conv2d3x3: 2-D 3x3 convolution.  

```C
//3x3 convolution layer
//output one feature map per kernel
__kernel void conv2d3x3(
	const int input_channels, const int input_size,
	const int pad, const int stride,
	const int start_channel, //start_channel is for 1x1 feature map in fire layer
	const int output_size,
	__global float* input_im,
	__global const float* filter_weight,
	__global const float* filter_bias,
	__global float *restrict output_im
	)
{
	int filter_index = get_global_id(0); //get output channel index
	int i =  get_global_id(1);

	filter_weight += filter_index * input_channels * 9;
	float bias = filter_bias[filter_index];
	output_im += (start_channel + filter_index) * output_size * output_size;
	
	//loop over output feature map
	//for(int i = 0; i < output_size; i++)
	{
		for(int j = 0; j < output_size; j++)
		{
			//compute one element in the output feature map
			float tmp = bias;
			
			//compute dot product of 2 input_channels x 3 x 3 matrix
			for(int k = 0; k < input_channels; k++)
			{
				#pragma unroll
				for(int l = 0; l < 3; l++)
				{
					int h = i * stride + l - pad;
					for(int m = 0; m < 3; m++)
					{
						int w = j * stride + m - pad;
						if((h >= 0) && (h < input_size) && (w >= 0) && (w < input_size))
						{
							tmp += input_im[k * input_size * input_size + h * input_size + w] \
                               * filter_weight[9 * k + 3 * l + m];
						}
					}
				}
			}

			//add relu activation after conv
			output_im[i * output_size + j] = (tmp > 0.0) ? tmp : 0.0;
		}
	}
}
```
#### OpenCL kernel: conv3x3_ST.cl

conv2d3x3: 2-D 3x3 convolution. 

```C
//3x3 convolution layer
//output one feature map per kernel
__kernel void conv2d3x3(
	const int input_channels, const int input_size,
	const int pad, const int stride,
	const int start_channel, //start_channel is for 1x1 feature map in fire layer
	const int output_size,
    const int filter_size,
	__global float* input_im,
	__global const float* filter_weight,
	__global const float* filter_bias,
	__global float *restrict output_im
	)
{
	
	//filter_weight += filter_index * input_channels * 9;
	output_im += start_channel * output_size * output_size;
	
	//loop over output feature map
	for(int filter_index = 0; filter_index < filter_size; filter_index++)
	{
        float bias = filter_bias[filter_index];

		for(int i = 0; i < output_size; i++)
		{
            for(int j = 0; j < output_size; j++)
            {
                //compute one element in the output feature map
                float tmp = bias;

                //compute dot product of 2 input_channels x 3 x 3 matrix
                for(int k = 0; k < input_channels; k++)
                {
                    #pragma unroll
                    for(int l = 0; l < 3; l++)
                    {
                        int h = i * stride + l - pad;
                        for(int m = 0; m < 3; m++)
                        {
                            int w = j * stride + m - pad;
                            if((h >= 0) && (h < input_size) && (w >= 0) && (w < input_size))
                            {
                                tmp += input_im[k * input_size * input_size + h * input_size + w] \
                                   * filter_weight[9 * k + 3 * l + m];
                            }
                        }
                    }
                }

                //add relu activation after conv
                output_im[i * output_size + j] = (tmp > 0.0) ? tmp : 0.0;                 
            }
		}
        
        filter_weight += input_channels * 9;
        output_im += output_size * output_size;
	}
}
```

Run OpenCL implement  

In [45]:
tamanyo=56 #input_size
canales_iniciales=64 #input_channels
canales_contraidos=16 #filter_size
canales_finales = 128

acumulado_pytorch=0

imagen = np.random.randint(10,size=(1,canales_contraidos, tamanyo, tamanyo)).astype(np.float32)
#imagen = np.ones((1,canales_contraidos, tamanyo, tamanyo)).astype(np.float32)

weights1=np.random.randint(10,size=(canales_iniciales, canales_contraidos, 3, 3)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_iniciales,)).astype(np.float32)

#weights1=np.ones((canales_iniciales, canales_contraidos, 3, 3)).astype(np.float32)
#bias1=np.ones((canales_iniciales,)).astype(np.float32)

squeeze_activation = nn.ReLU(inplace=True)

tic=pc()
squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=3, bias=False, padding=1)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida1_a_numpy=salida1_activation.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [46]:
# NDRANGE

h_sample = imagen.reshape(-1).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_expand3x3_weight = weights1.reshape(-1)
fire1_expand3x3_bias = bias1

d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

#offset = np.int32(canales_iniciales)
offset = np.int32(0)

tic2 = pc()

conv3x3_NDR(queue,(canales_iniciales, tamanyo), None, canales_contraidos, tamanyo, 1, 1, offset, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2


In [47]:
# Simple task

h_sample = imagen.reshape(-1).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_expand3x3_weight = weights1.reshape(-1)
fire1_expand3x3_bias = bias1

d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

#offset = np.int32(canales_iniciales)
offset = np.int32(0)

tic3 = pc()

conv3x3_ST(queue,(1,), None, np.int32(canales_contraidos), tamanyo, 1, 1, offset, tamanyo, canales_iniciales, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos1 = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3


In [48]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task)=",rtime1)

comparativa1=np.allclose(salida1_a_numpy, veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida1_a_numpy, veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(veamos, veamos1,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task): ",comparativa2)
print("comparativa (NDRange == Simple Task): ",comparativa3)

tiempo en segundos con pytorch=  0.23023968800043804
tiempo en segundos con opencl (NDRANGE)= 0.03479939099997864
tiempo en segundos con opencl (Simple Task)= 0.014478344000053767
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task):  True
comparativa (NDRange == Simple Task):  True


In [71]:
for i in range(canales_contraidos):
    for j in range(tamanyo):
        for k in range(tamanyo):
            if (abs(salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos1[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos1[i][j][k])

#### Step 3: emulation
Creat & build program

In [49]:
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[1])])
device = platforms[1].get_devices()

queue = cl.CommandQueue(context)

context

<pyopencl.Context at 0x67691f8 on <pyopencl.Device 'Intel(R) FPGA Emulation Device' on 'Intel(R) FPGA Emulation Platform for OpenCL(TM)' at 0x4f89ba8>>

In [52]:
wksp = '../device/v1.3/conv3x3/bin_em/'

file_dir = wksp + 'conv3x3_NDRange.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_NDR = cl.Program(context, device, [kernelSource]).build()

file_dir = wksp + 'conv3x3_ST.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_ST = cl.Program(context, device, [kernelSource]).build()

Creat kernels

In [53]:
conv3x3_NDR = program_NDR.conv2d3x3
conv3x3_NDR.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

conv3x3_ST = program_ST.conv2d3x3
conv3x3_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])


Run OpenCL implement  

In [54]:
tamanyo=56 #input_size
canales_iniciales=64 #input_channels
canales_contraidos=16 #filter_size
canales_finales = 128

acumulado_pytorch=0

imagen = np.random.randint(10,size=(1,canales_contraidos, tamanyo, tamanyo)).astype(np.float32)
#imagen = np.ones((1,canales_contraidos, tamanyo, tamanyo)).astype(np.float32)

weights1=np.random.randint(10,size=(canales_iniciales, canales_contraidos, 3, 3)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_iniciales,)).astype(np.float32)

#weights1=np.ones((canales_iniciales, canales_contraidos, 3, 3)).astype(np.float32)
#bias1=np.ones((canales_iniciales,)).astype(np.float32)

squeeze_activation = nn.ReLU(inplace=True)

tic=pc()
squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=3, bias=False, padding=1)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida1_a_numpy=salida1_activation.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [55]:
# NDRANGE

h_sample = imagen.reshape(-1).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_expand3x3_weight = weights1.reshape(-1)
fire1_expand3x3_bias = bias1

d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

#offset = np.int32(canales_iniciales)
offset = np.int32(0)

tic2 = pc()

conv3x3_NDR(queue,(canales_iniciales, tamanyo), None, canales_contraidos, tamanyo, 1, 1, offset, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2


In [56]:
# Simple task

h_sample = imagen.reshape(-1).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_expand3x3_weight = weights1.reshape(-1)
fire1_expand3x3_bias = bias1

d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

#offset = np.int32(canales_iniciales)
offset = np.int32(0)

tic3 = pc()

conv3x3_ST(queue,(1,), None, np.int32(canales_contraidos), tamanyo, 1, 1, offset, tamanyo, canales_iniciales, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos1 = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3


In [57]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task)=",rtime1)

comparativa1=np.allclose(salida1_a_numpy, veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida1_a_numpy, veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(veamos, veamos1,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task): ",comparativa2)
print("comparativa (NDRange == Simple Task): ",comparativa3)

tiempo en segundos con pytorch=  0.0030234180003390065
tiempo en segundos con opencl (NDRANGE)= 0.03991814599976351
tiempo en segundos con opencl (Simple Task)= 0.04235464699922886
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task):  True
comparativa (NDRange == Simple Task):  True


In [155]:
for i in range(canales_contraidos):
    for j in range(tamanyo):
        for k in range(tamanyo):
            if (abs(salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos1[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos1[i][j][k])

#### Step 4: simulación
Creat & build program

In [58]:
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[2])])
device = platforms[2].get_devices()

queue = cl.CommandQueue(context)

context

<pyopencl.Context at 0x8e027b0 on <pyopencl.Device 'SimulatorDevice : Multi-process Simulator (aclmsim0)' on 'Intel(R) FPGA SDK for OpenCL(TM)' at 0x7f5963ab50d8>>

In [59]:
wksp = '../device/v1.3/conv3x3/bin_sim/'

file_dir = wksp + 'conv3x3_NDRange.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_NDR = cl.Program(context, device, [kernelSource]).build()

file_dir = wksp + 'conv3x3_ST.aocx'

kernelSource = open(file_dir, mode='rb').read()
program_ST = cl.Program(context, device, [kernelSource]).build()

Creat kernels

In [61]:
conv3x3_NDR = program_NDR.conv2d3x3
conv3x3_NDR.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])

conv3x3_ST = program_ST.conv2d3x3
conv3x3_ST.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, None, None, None, None])


Run OpenCL implement  

In [62]:
tamanyo=5 #input_size
canales_iniciales=8 #input_channels
canales_contraidos=2 #filter_size
canales_finales = canales_iniciales

acumulado_pytorch=0

imagen = np.random.randint(10,size=(1,canales_contraidos, tamanyo, tamanyo)).astype(np.float32)
#imagen = np.ones((1,canales_contraidos, tamanyo, tamanyo)).astype(np.float32)

weights1=np.random.randint(10,size=(canales_iniciales, canales_contraidos, 3, 3)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_iniciales,)).astype(np.float32)

#weights1=np.ones((canales_iniciales, canales_contraidos, 3, 3)).astype(np.float32)
#bias1=np.ones((canales_iniciales,)).astype(np.float32)

squeeze_activation = nn.ReLU(inplace=True)

tic=pc()
squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=3, bias=False, padding=1)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida1_a_numpy=salida1_activation.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [63]:
# NDRANGE

h_sample = imagen.reshape(-1).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_expand3x3_weight = weights1.reshape(-1)
fire1_expand3x3_bias = bias1

d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

#offset = np.int32(canales_iniciales)
offset = np.int32(0)

tic2 = pc()

conv3x3_NDR(queue,(canales_iniciales, tamanyo), None, canales_contraidos, tamanyo, 1, 1, offset, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2

/tmp/ipykernel_4523/1417617651.py:12: RuntimeWarning: overflow encountered in cast
  h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance


In [64]:
# Simple task

h_sample = imagen.reshape(-1).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_expand3x3_weight = weights1.reshape(-1)
fire1_expand3x3_bias = bias1

d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)

#offset = np.int32(canales_iniciales)
offset = np.int32(0)

tic3 = pc()

conv3x3_ST(queue,(1,), None, np.int32(canales_contraidos), tamanyo, 1, 1, offset, tamanyo, canales_iniciales, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)

queue.finish()

veamos1 = h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3


/tmp/ipykernel_4523/1696061528.py:12: RuntimeWarning: overflow encountered in cast
  h_result_fire1_expand = np.empty(1 * canales_iniciales * tamanyo * tamanyo).astype(np.float32) # we wil only check 3x3 convolituional performance


In [65]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task)=",rtime1)

comparativa1=np.allclose(salida1_a_numpy, veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida1_a_numpy, veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(veamos, veamos1,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task): ",comparativa2)
print("comparativa (NDRange == Simple Task): ",comparativa3)

tiempo en segundos con pytorch=  0.0005055000001448207
tiempo en segundos con opencl (NDRANGE)= 42.37730355199983
tiempo en segundos con opencl (Simple Task)= 339.99705950299995
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task):  True
comparativa (NDRange == Simple Task):  True


In [15]:
print("pytorch", veamos.shape, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[0][0])
print("NDRange", veamos[0][0])
print("Simple task", veamos1[0][0])
print(imagen.shape, np.allclose(imagen, np.ones((1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32), rtol=1e-01, atol=1e-01))
#print(weights1[6])
# print(fire1_squeeze_weight)
#print(bias1[6])
# print(imagen1_obtenida)
# print(veamos2)
print(np.allclose(salida1_a_numpy, veamos1,rtol=1e-01, atol=1e-01))

pytorch (2, 5, 5) [123. 151. 184. 152. 138.]
NDRange [123. 151. 184. 152. 138.]
Simple task [ 77.  71. 126. 184. 132.]
(1, 8, 5, 5) False
False


In [78]:
for i in range(canales_contraidos):
    for j in range(tamanyo):
        for k in range(tamanyo):
            if (abs(salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos1[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos1[i][j][k])